In [1]:
import joblib
import os
import pandas as pd
import numpy as np
import random
import pickle
import hashlib
os.chdir('Resources/')

In [2]:
model = joblib.load('9_RF_Isolation_Model.joblib')
scaler = joblib.load('6_scaler.joblib')

In [3]:
def power(a, b, c):
    x = 1
    y = a
    while b > 0:
        if b % 2 != 0:
            x = (x * y) % c
        y = (y * y) % c
        b = int(b / 2)
    return x % c

In [10]:
def encrypt(msg, q, h, g):
    en_msg = []
    k = gen_key(q)  # Private key for sender
    s = power(h, k, q)
    p = power(g, k, q)
    
    for i in range(0, len(msg)):
        en_msg.append(s * msg[i])  # Encrypt each element
    
    return en_msg, p

In [5]:
public_key_file_path = '3_Public_Key.pkl'

with open(public_key_file_path, 'rb') as public_key_file:
    column_keys = pickle.load(public_key_file)

In [6]:
df_h = pd.read_csv('6_Preprocessed_Data.csv')
df_s = pd.read_csv('1_Structured_Data.csv')

In [7]:
X = df_s.drop(['HeartDisease'], axis='columns')
Y = df_s[['HeartDisease']]

In [13]:
count = 0
mismatched = []

# Assuming X, df_h, column_keys, scaler, and model are already defined

for i in range(4044):
    row_index = i

    # Check if the row doesn't contain missing values
    if X.iloc[row_index].notnull().all():
        encrypted_row = X.iloc[row_index].copy()

        for column in X.columns:
            if column in column_keys:  # Check if the column has an entry in column_keys
                q, h, g = column_keys[column]  # Retrieve the public key components for the current column
                value = X.at[row_index, column]  # Get the value of the current column for the row

                # Check if the value is a string
                if isinstance(value, str):
                    # Convert string to numeric by summing the ordinal values of each character
                    numeric_value = sum(ord(char) for char in value)
                    encrypted_value, _ = encrypt(numeric_value, q, h, g)  # Encrypt the numeric value
                else:
                    # If the value is numeric (float or int), encrypt it
                    encrypted_value, _ = encrypt(float(value), q, h, g)

                # Store the encrypted value in the encrypted_row
                encrypted_row[column] = encrypted_value  # Ensure the value fits within a certain range
            else:
                print(f"Warning: Missing key for column {column}")
                encrypted_row[column] = None  # Or handle the missing key case differently

        # Create a new DataFrame with the encrypted row
        encrypted_row_df = pd.DataFrame([encrypted_row])

        # Scale the encrypted row using the scaler
        X_new_scaled_s = scaler.transform(encrypted_row_df)

        # Predict the label based on the scaled encrypted values
        pred_new_s = model.predict(X_new_scaled_s)

        # Extract the original row (excluding the label column)
        X_new_h = df_h.iloc[row_index, :-1]
        X_new_h = np.array(X_new_h).reshape(1, -1)

        # Predict the label based on the original (unencrypted) data
        pred_new_h = model.predict(X_new_h)

        # Check if the predictions from the encrypted and original data are different
        if pred_new_h[0] != pred_new_s[0]:
            count += 1
            mismatched.append(row_index + 2)  # Append the row number (adjusted for 1-based indexing)

        # Print the predicted labels and the mismatch count
        print(f"Predicted label for {row_index + 2}th data: ({pred_new_h[0]}, {pred_new_s[0]})", count)

# Print the final mismatch count and indices
print(f"Total mismatches: {count}")
print(f"Rows with mismatches: {mismatched}")


Predicted label for 2th data: (0, 0) 0
Predicted label for 3th data: (1, 0) 1
Predicted label for 4th data: (0, 0) 1
Predicted label for 5th data: (1, 0) 2
Predicted label for 6th data: (0, 0) 2
Predicted label for 7th data: (0, 0) 2
Predicted label for 8th data: (0, 0) 2
Predicted label for 9th data: (0, 0) 2
Predicted label for 10th data: (1, 0) 3
Predicted label for 11th data: (0, 0) 3
Predicted label for 12th data: (0, 0) 3
Predicted label for 13th data: (1, 0) 4
Predicted label for 14th data: (0, 0) 4
Predicted label for 15th data: (1, 0) 5
Predicted label for 16th data: (0, 0) 5
Predicted label for 17th data: (0, 0) 5
Predicted label for 18th data: (1, 0) 6
Predicted label for 19th data: (0, 0) 6
Predicted label for 20th data: (1, 0) 7


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

Predicted label for 21th data: (1, 0) 8
Predicted label for 22th data: (0, 0) 8
Predicted label for 23th data: (0, 0) 8
Predicted label for 24th data: (0, 0) 8
Predicted label for 25th data: (1, 0) 9
Predicted label for 26th data: (0, 0) 9
Predicted label for 27th data: (0, 0) 9
Predicted label for 28th data: (0, 0) 9
Predicted label for 29th data: (0, 0) 9
Predicted label for 30th data: (0, 0) 9
Predicted label for 31th data: (0, 0) 9
Predicted label for 32th data: (1, 0) 10
Predicted label for 33th data: (0, 0) 10
Predicted label for 34th data: (1, 0) 11
Predicted label for 35th data: (1, 0) 12
Predicted label for 36th data: (0, 0) 12
Predicted label for 37th data: (0, 0) 12
Predicted label for 38th data: (1, 0) 13
Predicted label for 39th data: (0, 0) 13
Predicted label for 40th data: (0, 0) 13
Predicted label for 41th data: (0, 0) 13


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

Predicted label for 42th data: (0, 0) 13
Predicted label for 43th data: (1, 0) 14
Predicted label for 44th data: (0, 0) 14
Predicted label for 45th data: (0, 0) 14
Predicted label for 46th data: (1, 0) 15
Predicted label for 47th data: (0, 0) 15
Predicted label for 48th data: (0, 0) 15
Predicted label for 49th data: (0, 0) 15
Predicted label for 50th data: (1, 0) 16
Predicted label for 51th data: (1, 0) 17
Predicted label for 52th data: (1, 0) 18
Predicted label for 53th data: (1, 0) 19
Predicted label for 54th data: (0, 0) 19
Predicted label for 55th data: (0, 0) 19
Predicted label for 56th data: (0, 0) 19
Predicted label for 57th data: (0, 0) 19
Predicted label for 58th data: (1, 0) 20
Predicted label for 59th data: (1, 0) 21
Predicted label for 60th data: (0, 0) 21
Predicted label for 61th data: (1, 0) 22
Predicted label for 62th data: (0, 0) 22
Predicted label for 63th data: (0, 0) 22
Predicted label for 64th data: (0, 0) 22


...

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

Predicted label for 4004th data: (0, 0) 1825
Predicted label for 4005th data: (1, 0) 1826
Predicted label for 4006th data: (0, 0) 1826
Predicted label for 4007th data: (1, 0) 1827
Predicted label for 4008th data: (1, 0) 1828
Predicted label for 4009th data: (0, 0) 1828
Predicted label for 4010th data: (1, 0) 1829
Predicted label for 4011th data: (1, 0) 1830
Predicted label for 4012th data: (0, 0) 1830
Predicted label for 4013th data: (1, 0) 1831
Predicted label for 4014th data: (1, 0) 1832
Predicted label for 4015th data: (1, 0) 1833
Predicted label for 4016th data: (0, 0) 1833


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

Predicted label for 4017th data: (1, 0) 1834
Predicted label for 4018th data: (1, 0) 1835
Predicted label for 4019th data: (1, 0) 1836
Predicted label for 4020th data: (1, 0) 1837
Predicted label for 4021th data: (0, 0) 1837
Predicted label for 4022th data: (1, 0) 1838
Predicted label for 4023th data: (1, 0) 1839
Predicted label for 4024th data: (1, 0) 1840
Predicted label for 4025th data: (1, 0) 1841
Predicted label for 4026th data: (0, 0) 1841
Predicted label for 4027th data: (1, 0) 1842
Predicted label for 4028th data: (0, 0) 1842
Predicted label for 4029th data: (0, 0) 1842
Predicted label for 4030th data: (1, 0) 1843
Predicted label for 4031th data: (1, 0) 1844
Predicted label for 4032th data: (0, 0) 1844
Predicted label for 4033th data: (0, 0) 1844
Predicted label for 4034th data: (1, 0) 1845
Predicted label for 4035th data: (1, 0) 1846
Predicted label for 4036th data: (1, 0) 1847
Predicted label for 4037th data: (0, 0) 1847
Predicted label for 4038th data: (1, 0) 1848
Predicted 

c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feat

In [ ]:
print("No. of mismatched labels:", count)

No. of mismatched labels: 0


In [ ]:
print(mismatched)

[]
